In [1]:
!pip install gradio

  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached pydantic-2.10.5-py3-none-any.whl.metadata (30 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached starlette-0.45.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  U

In [6]:
import os
import gradio as gr

def load_labels(folder):
    """Đọc nhãn từ các file trong thư mục."""
    labels = {}
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(".txt") or file.endswith(".lg"):
                filepath = os.path.join(root, file)
                with open(filepath, "r", encoding="utf-8") as f:
                    labels[file] = f.read().strip()
    for label in labels:
        labels[label] = labels[label][1:]
    return labels

def compare_labels(result_folder, folders_to_compare):
    """So sánh nhãn trong result với các thư mục khác."""
    result_labels = load_labels(result_folder)
    comparison_labels = {}

    for folder in folders_to_compare:
        comparison_labels.update(load_labels(folder))
    
    mismatched_labels = []
    for file, result_label in result_labels.items():
        if file in comparison_labels:
            if result_label != comparison_labels[file]:
                mismatched_labels.append({
                    "file": file,
                    "result_label": result_label,
                    "expected_label": comparison_labels[file],
                })
        else:
            mismatched_labels.append({
                "file": file,
                "result_label": result_label,
                "expected_label": "Not found",
            })
    
    return mismatched_labels

def display_comparison(result_folder, folder_2014, folder_2016, folder_2019):
    """Kết quả so sánh để hiển thị trên giao diện."""
    folders_to_compare = [folder_2014, folder_2016, folder_2019]
    mismatched_labels = compare_labels(result_folder, folders_to_compare)
    
    if not mismatched_labels:
        return "Tất cả các nhãn đều khớp!"
    
    results = "Các nhãn không khớp:\n\n"
    for mismatch in mismatched_labels:
        results += (
            f"File: {mismatch['file']}\n"
            f"Nhãn trong result: {mismatch['result_label']}\n"
            f"Nhãn mong đợi: {mismatch['expected_label']}\n\n"
        )
    return results

# Giao diện Gradio
def main(result_folder, folder_2014, folder_2016, folder_2019):
    return display_comparison(result_folder, folder_2014, folder_2016, folder_2019)

interface = gr.Interface(
    fn=main,
    inputs=[
        gr.Textbox(label="Thư mục result", placeholder="Nhập đường dẫn tới thư mục result"),
        gr.Textbox(label="Thư mục 2014", placeholder="Nhập đường dẫn tới thư mục 2014"),
        gr.Textbox(label="Thư mục 2016", placeholder="Nhập đường dẫn tới thư mục 2016"),
        gr.Textbox(label="Thư mục 2019", placeholder="Nhập đường dẫn tới thư mục 2019"),
    ],
    outputs="text",
    title="So sánh nhãn",
    description="So sánh nhãn trong thư mục result với nhãn trong các thư mục khác.",
)

if __name__ == "__main__":
    interface.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [ ]:
import os
import gradio as gr

def load_labels_from_caption(caption_file):
    """
    Đọc nhãn từ file caption.txt.
    Mỗi dòng trong file có định dạng: <file_name> <label>
    """
    labels = {}
    if os.path.exists(caption_file):
        with open(caption_file, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split("\t")
                if len(parts) == 2:
                    file_name, label = parts
                    labels[file_name] = label.strip()
    return labels

def load_labels_from_result(result_folder):
    """
    Đọc nhãn từ các file trong thư mục result.
    Bỏ phần đầu (%UN19_xxx) và ký tự $.
    """
    labels = {}
    for file in os.listdir(result_folder):
        if file.endswith(".txt"):
            filepath = os.path.join(result_folder, file)
            with open(filepath, "r", encoding="utf-8") as f:
                lines = f.readlines()
                if len(lines) > 1:
                    # Lấy dòng thứ hai (nhãn) và loại bỏ phần không cần thiết
                    label = lines[1].strip()
                    if label.startswith("$") and label.endswith("$"):
                        label = label[1:-1]  # Bỏ ký tự $ đầu và cuối
                    labels[file] = label
    return labels


def compare_labels(result_folder, folders_to_compare):
    """
    So sánh nhãn trong result với nhãn trong các thư mục 2014, 2016, 2019.
    """
    result_labels = load_labels_from_result(result_folder)
    comparison_labels = {}

    # Duyệt qua từng thư mục và lấy nhãn từ caption.txt
    for folder in folders_to_compare:
        caption_file = os.path.join(folder, "caption.txt")
        comparison_labels.update(load_labels_from_caption(caption_file))

    mismatched_labels = []
    for file, result_label in result_labels.items():
        # Tên file trong thư mục result không bao gồm phần mở rộng
        file_base = os.path.splitext(file)[0]
        if file_base in comparison_labels:
            expected_label = comparison_labels[file_base]

            if result_label != expected_label:
                mismatched_labels.append({
                    "file": file,
                    "result_label": result_label,
                    "expected_label": expected_label,
                })
        else:
            mismatched_labels.append({
                "file": file,
                "result_label": result_label,
                "expected_label": "Not found",
            })

    return mismatched_labels

def display_comparison(result_folder, folder_2014, folder_2016, folder_2019):
    """
    Hiển thị kết quả so sánh nhãn.
    """
    folders_to_compare = [folder_2014, folder_2016, folder_2019]
    mismatched_labels = compare_labels(result_folder, folders_to_compare)

    if not mismatched_labels:
        return "Tất cả các nhãn đều khớp!"

    results = "Các nhãn không khớp:\n\n"
    results += f"Số lượng: {len(mismatched_labels)}\n\n"
    for mismatch in mismatched_labels:
        results += (
            f"File: {mismatch['file']}\n"
            f"Nhãn trong result: {mismatch['result_label']}\n"
            f"Nhãn mong đợi: {mismatch['expected_label']}\n\n"
        )
        
    return results

# Giao diện Gradio
def main(result_folder, folder_2014, folder_2016, folder_2019):
    return display_comparison(result_folder, folder_2014, folder_2016, folder_2019)

interface = gr.Interface(
    fn=main,
    inputs=[
        gr.Textbox(label="Thư mục result", placeholder="Nhập đường dẫn tới thư mục result"),
        gr.Textbox(label="Thư mục 2014", placeholder="Nhập đường dẫn tới thư mục 2014"),
        gr.Textbox(label="Thư mục 2016", placeholder="Nhập đường dẫn tới thư mục 2016"),
        gr.Textbox(label="Thư mục 2019", placeholder="Nhập đường dẫn tới thư mục 2019"),
    ],
    outputs="text",
    title="So sánh nhãn",
    description="So sánh nhãn trong thư mục result với nhãn trong các file caption.txt.",
)

if __name__ == "__main__":
    interface.launch()


* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
